In [3]:
import ee
import geemap
import geeft

In [4]:
ee.Initialize()

In [5]:
aoi = ee.Geometry.Point([-77.3828, 44.1628])

In [11]:
def cloud_mask(image):
    qa = image.select("QA60")
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions.
    mask = (
        qa.bitwiseAnd(cloud_bit_mask)
        .eq(0)
        .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    )

    return image.updateMask(mask)


def add_ndvi(image):
    return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI'))

In [12]:
dataset = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterBounds(aoi)
    .filterDate('2018', '2022')
    .map(cloud_mask)
    .map(add_ndvi)
)

dataset.first()

In [13]:
ft = geeft.compute_fourier_transform(dataset=dataset, dependent='NDVI')

In [14]:
Map = geemap.Map()

Map.centerObject(aoi, 10)
Map.addLayer(ft.select('phase_1'))
Map.addLayerControl()
Map

Map(center=[44.16279999999999, -77.3828], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…